In [23]:
import yfinance as yf
import pandas as pd

df = yf.download('BTC-USD', start='2025-01-01', end='2025-09-22', interval='1wk')


/tmp/ipykernel_599072/2684615471.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('BTC-USD', start='2025-01-01', end='2025-09-22', interval='1wk')
[*********************100%***********************]  1 of 1 completed


In [24]:
data = df.copy()

In [25]:
print(df.head())
df["Date"] = pd.to_datetime(df.index, utc=True)
print(df.head())

Price               Close           High           Low           Open  \
Ticker            BTC-USD        BTC-USD       BTC-USD        BTC-USD   
Date                                                                    
2025-01-01   96922.703125  102712.484375  92788.125000   93425.101562   
2025-01-08   96534.046875   97352.664062  89260.101562   96924.164062   
2025-01-15  106146.265625  109114.882812  96501.640625   96534.046875   
2025-01-22  101332.476562  107098.546875  97795.937500  106136.382812   
2025-01-29   97871.820312  106418.765625  91242.890625  101317.523438   

Price             Volume  
Ticker           BTC-USD  
Date                      
2025-01-01  259517878648  
2025-01-08  355206513683  
2025-01-15  526047818939  
2025-01-22  392990612048  
2025-01-29  414333348359  
Price               Close           High           Low           Open  \
Ticker            BTC-USD        BTC-USD       BTC-USD        BTC-USD   
Date                                                 

In [26]:
df["Date"] = pd.to_datetime(df["Date"]).astype(int) / 10**9


In [27]:

df.reset_index(drop=True, inplace=True)


In [28]:
df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
df.to_csv("dataset/cryptex/candlesticks-W-full.csv", index=False)

In [34]:
df = pd.read_csv("dataset/cryptex/candlesticks-W-2025.csv")


In [35]:
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
df.columns = ["timestamp", "open", "high", "low", "close", "volume"]

df.to_csv("dataset/cryptex/candlesticks-W-2025.csv", index=False)


TypeError: cannot convert the series to <class 'int'>

In [ ]:
df = pd.read_csv("dataset/cryptex/candlesticks-W-2014-2024.csv")

'hi there'